<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/EEG(seed)GAN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!kaggle datasets download -d daviderusso7/seed-dataset
!unzip seed-dataset.zip -d /content/SEED


Dataset URL: https://www.kaggle.com/datasets/daviderusso7/seed-dataset
License(s): unknown
seed-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  seed-dataset.zip
replace /content/SEED/DatasetCaricatoNoImage.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/SEED/DatasetCaricatoNoImage.npz  
replace /content/SEED/LabelsNoImage.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/SEED/LabelsNoImage.npz  
replace /content/SEED/SubjectsNoImage.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/SEED/SubjectsNoImage.npz  


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load EEG data
data = np.load("/content/SEED/DatasetCaricatoNoImage.npz")
labels = np.load("/content/SEED/LabelsNoImage.npz")

X = data['arr_0']  # EEG features (50910, 5, 62)
y = labels['arr_0']  # Emotion labels (50910,)

# Encode labels (convert emotions to numbers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert emotions to integers

# Normalize EEG features
scaler = StandardScaler()
X_reshaped = X.reshape(-1, 62)  # Flatten time steps for scaling
X_scaled = scaler.fit_transform(X_reshaped)
X = X_scaled.reshape(50910, 5, 62)  # Reshape back

# Split into Train & Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")


Train Shape: (40728, 5, 62), Test Shape: (10182, 5, 62)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, Flatten, LeakyReLU

# Generator Model
def build_generator(latent_dim):
    model = Sequential([
        Dense(128, activation="relu", input_dim=latent_dim),
        Dense(310, activation="relu"),  # 5 * 62 = 310
        Reshape((5, 62))
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=(5, 62)),
        Dense(128, activation=LeakyReLU(0.2)),
        Dense(1, activation="sigmoid")  # Binary classification (Real vs. Fake)
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [8]:
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# GAN Model: Combine Generator & Discriminator
gan = Sequential([generator, discriminator])
discriminator.trainable = False  # Train only the generator when using GAN
gan.compile(loss="binary_crossentropy", optimizer="adam")

# Train GAN
epochs = 1000
batch_size = 32

for epoch in range(epochs):
    # Select real EEG samples
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_eeg = X_train[idx]

    # Generate fake EEG samples
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_eeg = generator.predict(noise)

    # Train Discriminator
    d_loss_real = discriminator.train_on_batch(real_eeg, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_eeg, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]:.4f}, G Loss: {g_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, D Loss: 0.5744, G Loss: 0.5874
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━

In [9]:
synthetic_samples = generator.predict(np.random.normal(0, 1, (10000, latent_dim)))  # Generate 10,000 samples

# Assign random labels to synthetic data
synthetic_labels = np.random.choice(y_encoded, 10000)

# Merge Real and Synthetic EEG Data
X_final = np.vstack((X_train, synthetic_samples))
y_final = np.concatenate((y_train, synthetic_labels))

print(f"Final Training Data Shape: {X_final.shape}, Final Labels Shape: {y_final.shape}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Final Training Data Shape: (50728, 5, 62), Final Labels Shape: (50728,)


In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Define LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(5, 62)),
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')  # Multi-class output
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_final, y_final, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.6497 - loss: 0.6892 - val_accuracy: 0.9327 - val_loss: 0.1855
Epoch 2/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.8259 - loss: 0.3239 - val_accuracy: 0.9681 - val_loss: 0.0885
Epoch 3/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8511 - loss: 0.2671 - val_accuracy: 0.9951 - val_loss: 0.0171
Epoch 4/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.8550 - loss: 0.2545 - val_accuracy: 0.9893 - val_loss: 0.0309
Epoch 5/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8634 - loss: 0.2352 - val_accuracy: 0.9898 - val_loss: 0.0338
Epoch 6/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.8631 - loss: 0.2340 - val_accuracy: 1.0000 - val_loss: 6.9702e-04
Epoch 7/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8658 - loss: 0.2283 - val_accuracy: 1.0000 - val_loss: 8.2069e-04
Epoch 8/10
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 

In [13]:
from sklearn.metrics import accuracy_score, classification_report

# Predict emotions on test data
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Convert back to emotion names
predicted_emotions = label_encoder.inverse_transform(predicted_labels)

# Model Accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Convert integer-encoded labels back to string labels
target_names = [str(cls) for cls in label_encoder.classes_]

# Generate classification report
print("\nClassification Report:\n", classification_report(y_test, predicted_labels, target_names=target_names))


319/319 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Model Accuracy: 100.00%

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3301
           1       1.00      1.00      1.00      3372
           2       1.00      1.00      1.00      3509

    accuracy                           1.00     10182
   macro avg       1.00      1.00      1.00     10182
weighted avg       1.00      1.00      1.00     10182



In [17]:
model.save('/content/drive/MyDrive/lstm_emotion_model(seed).h5')  # Save in Google Drive
print("Model saved successfully!")


Model saved successfully!


In [18]:
from keras.models import load_model

# Load the trained LSTM model
lstm_model = load_model("/content/drive/MyDrive/lstm_emotion_model(seed).h5")  # Update the correct path


In [22]:
import numpy as np

# Example new EEG input (random values for testing)
new_eeg_input = np.random.randn(1, 5, 62)  # Replace with actual EEG data
print(new_eeg_input.shape)
print(new_eeg_input)

# Normalize using the same scaler (if saved, load it)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
new_eeg_input = scaler.fit_transform(new_eeg_input.reshape(-1, 62)).reshape(1, 5, 62)


(1, 5, 62)
[[[-7.53967054e-01  1.02388383e+00 -1.20731919e+00  2.12484750e-01
   -6.62842840e-02 -8.51256150e-01 -7.40273602e-01 -6.48679756e-01
   -1.34336665e+00 -3.34227119e-01 -1.93506656e+00 -5.43593063e-01
   -4.84970967e-01  4.38208988e-01 -3.30240778e-01 -2.71472626e-01
   -2.42865846e-01  1.44299511e-01 -4.58215024e-02  5.64496337e-01
   -2.28121767e-01  2.39358112e-01  2.55416668e-01  1.32862290e+00
   -1.39270140e-01 -1.17932108e-01  2.53592613e-01 -1.55437680e+00
    4.51165892e-01 -1.59583707e+00  4.73247372e-01  1.48715361e+00
   -8.72950970e-01 -1.48209186e+00  1.39740294e+00  1.49272683e+00
    9.38552885e-01 -6.03614902e-01 -4.60351375e-01  5.23316818e-01
    1.39698127e+00 -1.81736448e+00 -1.05294743e-01  4.00021436e-01
    2.88578134e-01 -6.45912619e-01  1.30219647e+00 -5.01948718e-01
   -6.13406375e-01  4.92789743e-01  3.22494792e-01 -9.57695454e-02
   -6.62065212e-02  6.94336118e-02 -1.15843790e+00  5.61295015e-01
   -2.87086836e-01  6.18695103e-02 -2.22179377e+00 

In [23]:
# Predict using the trained LSTM model
emotion_prediction = lstm_model.predict(new_eeg_input)  # Output shape: (1, 3) if 3 emotions

# Get the index with the highest probability
predicted_index = np.argmax(emotion_prediction)

# Define emotion labels (make sure they match training labels)
emotion_labels = ["Positive", "Neutral", "Negative"]  # Update if different

# Map index to emotion
predicted_emotion = emotion_labels[predicted_index]

print(f"Predicted Emotion: {predicted_emotion}")
print("Raw Model Output:", emotion_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted Emotion: Positive
Raw Model Output: [[0.8918256  0.03532846 0.07284598]]
